# Laboratorio 2 - Agrupación

### Integrantes

1. Cesar Avellaneda, 202214746, c.avellanedac@uniandes.edu.co.
2. Santiago Tinjaca, 202215991, s.tinjaca@uniandes.edu.co.
3. Jorge Bustamante, 202210581, j.bustamantep@uniandes.edu.co.

Este notebook tiene los siguientes elementos: 
1. Cargue de los datos.

2. Entendimiento de los datos: Describir las características más relevantes de los datos y todo el perfilamiento de datos, incluir el análisis de calidad de datos y hacer una preselección de las variables más importantes para la etapa de modelado.

3. Preparación de datos: Solucionar los problemas de calidad de datos previamente identificados que afecten el modelo a construir. Además, debe aplicar todos los proceso de preprocesamiento de datos necesarios para la construcción del modelo de agrupación.

4. Modelado: Utilizando las variables previamente seleccionadas, construir un modelo de agrupación que tenga siluetas cercanas a 1.

5. Evaluación cuantitativa: A partir de las métricas seleccionadas para evaluar y seleccionar el mejor modelo, explicar el resultado obtenido desde el punto de vista cuantitativo. Contestar a la pregunta: ¿Su equipo recomienda utilizar en producción el modelo de agrupación para optimizar tiempos? ¿Por qué? En caso de no recomendar el uso del modelo, ¿qué recomendaciones haría para continuar iterando con el objetivo de la construcción de un mejor modelo?

6. Evaluación cualitativa: Interpretación de los grupos obtenidos y relación entre estos y el objetivo de la organización. Verificar si es posible usar los datos obtenidos para optimizar la toma de decisiones.

### Entendimiento del negocio:
El caso de estudio es de un hospital que haciendo uso de la metodología KTAS quiere solicitar un modelo que pueda agrupar pacientes por ciertas características en común y le ayude al hospital a optimizar sus tiempos de respuesta.
### Enfoque Analítico:
En este laboratorio vamos a hacer un modelo predictivo usando un aprendizaje supervisado y un modelo de agrupación para hacer uso de las condiciones de llegada de los pacientes y optimizar las decisiones tomadas.

### 1. Carga de datos

In [230]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

In [231]:
datos=pd.read_csv('./data/202420_Laboratorio 2 - Agrupación_202420_Laboratorio_2_-_Agrupación_data.csv', sep=',', encoding = 'utf-8', index_col=0)

In [ ]:
datos.shape

In [ ]:
datos.dtypes

In [ ]:
datos.sample(5)

### 2. Entendimiento de los datos

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
def calcularEWS(registro):
    total = 0
    
    # Frecuencia respiratoria (RR)
    if registro['RR'] <= 8:
        total += 2
    elif 9 <= registro['RR'] <= 14:
        total += 0
    elif 15 <= registro['RR'] <= 20:
        total += 1
    elif 21 <= registro['RR'] <= 29:
        total += 2
    elif registro['RR'] >= 30:
        total += 3
    
    # Presión arterial sistólica (SBP)
    if registro['SBP'] <= 70:
        total += 3
    elif 71 <= registro['SBP'] <= 80:
        total += 2
    elif 81 <= registro['SBP'] <= 100:
        total += 1
    elif 101 <= registro['SBP'] <= 199:
        total += 0
    elif registro['SBP'] >= 200:
        total += 2

    # Frecuencia cardíaca (HR)
    if registro['HR'] <= 40:
        total += 2
    elif 41 <= registro['HR'] <= 50:
        total += 1
    elif 51 <= registro['HR'] <= 100:
        total += 0
    elif 101 <= registro['HR'] <= 110:
        total += 1
    elif 111 <= registro['HR'] <= 129:
        total += 2
    elif registro['HR'] >= 130:
        total += 3

    # Temperatura corporal (BT)
    if registro['BT'] < 35.0:
        total += 2
    elif 35.0 <= registro['BT'] <= 38.4:
        total += 0
    elif 38.5 <= registro['BT']:
        total += 2

    # Saturación de oxígeno (Saturacion)
    if registro['Saturacion'] <= 91:
        total += 3
    elif 92 <= registro['Saturacion'] <= 93:
        total += 2
    elif 94 <= registro['Saturacion'] <= 95:
        total += 1

    # Nivel de conciencia
    if registro['Estado_Mental'] == 1:
        total += 0
    elif registro['Estado_Mental'] == 2:
        total += 1
    elif registro['Estado_Mental'] == 3:
        total += 2
    elif registro['Estado_Mental'] == 4:
        total += 3
    return total

datos['EWS'] = datos.apply(calcularEWS, axis=1)
print(datos.head())

In [ ]:
#Visualización de todas las variables numéricas
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos, orient="v")

In [ ]:
name_cols_float = datos.select_dtypes(include = ['float']).columns
name_cols_int = datos.select_dtypes(include = ['int64']).columns
name_cols_int

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_float], orient="v")

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_int], orient="v")

In [ ]:
name_cols_non_number = datos.select_dtypes(include = ['object']).columns
name_cols_non_number

In [ ]:
datos.Queja_Principal.sample(10)

In [ ]:
datos[name_cols_non_number].describe()

In [ ]:
datos[name_cols_non_number].sample(15)

#### Análisis de completitud

In [ ]:
datos.isnull().sum() / datos.shape[0]

### 3. Preparación de datos

Dato que los datos atipicos de duración de instancia en minutos están bastante arriba a diferencia de los demás registros, decidimos escoger solo los datos que están por debajo de 850. 

Problemas de calidad:

Primero el dolor reportado por la enfermera, el cual tiene un 44% de sus entradas en null.
Nos dimos cuenta de que estas se correspondían cuando el paciente no tenía dolor, por lo que las asignamos a 0.

In [247]:
datos=datos[datos["Duracion_Estancia_Min"]<=850]

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_int], orient="v")

In [ ]:
name_cols = ['KTAS_experto', 'Estado_Mental' ,'dolor_NRS']
datos_cols_selec = datos[name_cols].copy()
datos_cols_selec.describe()


In [ ]:
datos_cols_selec['dolor_NRS'] = pd.to_numeric(datos_cols_selec['dolor_NRS'], errors='coerce')
datos_cols_selec['dolor_NRS'] = datos_cols_selec['dolor_NRS'].fillna(0)
datos_cols_selec = datos_cols_selec.reset_index(drop=True)
datos_cols_selec.describe()


In [ ]:
fig=plt.figure(figsize=(20,10))
ax = sns.boxplot(data=datos_cols_selec, orient="v")

In [ ]:
sns.pairplot(data=datos_cols_selec, hue="dolor_NRS")

### 4. Modelamiento

In [ ]:
mms = MinMaxScaler()


datos_prep_norm = datos_cols_selec.copy()
datos_prep_norm[name_cols] = mms.fit_transform(datos_cols_selec[name_cols])
datos_prep_norm=datos_prep_norm[name_cols]
saved_cols = datos_prep_norm.columns

datos_prep_norm = pd.DataFrame(datos_prep_norm, columns =saved_cols)
print(datos_prep_norm.head())

In [ ]:
datos_prep_norm.describe()

In [255]:
def plot_distortion(data,
                    k_min=1, 
                    k_max=11,
                    ylabel = 'Distortion',
                    xlabel = 'Number of clusters',
                    title = 'Distortion Plot'):
    '''
    Graficar el codo de los clusters
    
    Parametros
    ----------
    data : np.array
        El arreglo con los datos
    k_min : int
        Valor mínimo para k
    k_max : int
        Valor máximo para k
    xlabel : string
        La etiqueta del eje x
    ylabel  string
        La etiqueta del eje y    
    title : string
        El titulo de la gráfica  
    '''
    distortions = []
    for i in range(k_min, k_max):
        km = KMeans(n_clusters=i,
                 init='k-means++',
                 n_init=10,
                 max_iter=300,
                 random_state=0)
        km.fit(data)
        distortions.append(km.inertia_)
    plt.plot(range(k_min,k_max), distortions, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
plot_distortion(datos_prep_norm,1,11)

In [ ]:
N_clusters=2
kmeans = KMeans(n_clusters=N_clusters, random_state=0) 
kmeans = kmeans.fit(datos_prep_norm)

In [ ]:
labels = kmeans.labels_
datos_prep_norm['Cluster'] = labels

cluster_distrib = datos_prep_norm['Cluster'].value_counts()

fig=plt.figure(figsize=(12,8))
sns.barplot(x=cluster_distrib.index, y=cluster_distrib.values, color='b')

In [ ]:
cols_number = datos_prep_norm.to_numpy()
datos_prep_norm.groupby('Cluster').count()

In [ ]:
plt.scatter(cols_number[kmeans.labels_ == 0, 0], cols_number[kmeans.labels_ == 0, 1], s = 100, marker='v', c = 'red', label = 'Cluster 1')
plt.scatter(cols_number[kmeans.labels_ == 1, 0], cols_number[kmeans.labels_ == 1, 1], s = 100, marker='o', c = 'blue', label = 'Cluster 2')
plt.scatter(cols_number[kmeans.labels_ == 2, 0], cols_number[kmeans.labels_ == 2, 1], s = 100, marker='*', c = 'green', label = 'Cluster 3')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'yellow', label = 'Centroids')
plt.title('Clusters')
plt.xlabel(name_cols[0])
plt.ylabel(name_cols[1])
plt.legend()
plt.show()

In [268]:
def plot_silhouette(data, 
                    labels,
                   ylabel = 'Grupos',
                   xlabel = "Coeficiente de silueta",
                   title = 'Gráfica de silueta'):
    '''
    Graficar la silueta de los clusters
    
    Parametros
    ----------
    data : np.array
        El arreglo con los datos
    labels : np.array
        El arreglo con las etiquetas correspondientes
    ylabel  string
        La etiqueta del eje y
    xlabel : string
        La etiqueta del eje x
    title : string
        El titulo de la gráfica        
    '''
    cluster_labels = np.unique(labels)
    print(cluster_labels)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(data,
                                        labels,
                                        metric='euclidean')
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[labels == c]
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)
        color = cm.jet(float(i) / n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper),
                        c_silhouette_vals,
                        height=1.0,
                        edgecolor='none',
                        color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouette_vals)
    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg,
                color="red",
                linestyle="--")
    plt.yticks(yticks, cluster_labels+1)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.show()

In [ ]:
plot_silhouette(data = datos_prep_norm, 
                labels = kmeans.labels_, 
                ylabel = 'Modelo de dos Agrupaciones')